In [9]:
import numpy as np
import pickle
import os
from tqdm.notebook import tqdm

seed = 2023

In [10]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
from nlpsig_networks.scripts.swnu_network_functions import (
    swnu_network_hyperparameter_search,
)

In [12]:
output_dir = "talklife_moc_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

Talklife MoC

In [13]:
%run load_talklifemoc.py

In [14]:
%run load_sbert-embeddings.py

In [15]:
sbert_embeddings.shape

torch.Size([18604, 384])

SWNU Network

In [16]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
num_features = len(features)
include_features_in_path = True
include_features_in_input = False

In [17]:
#create indices for kfold
fold_col_names = [c for c in df.columns if 'fold' in c ]
fold_list = []
for foldc in fold_col_names:
    fold_list.append((df[df[foldc]=='train'].index, df[df[foldc]=='dev'].index, df[df[foldc]=='test'].index))
fold_list = tuple(fold_list)

In [18]:
num_epochs = 100
embedding_dim = 384
dimensions = [15]
dimreduction_method = ["umap"]
swnu_hidden_dim_sizes_and_sig_depths = [([12], 3), ([10], 4)]
ffn_hidden_dim_sizes = [[256, 256], [512, 512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [19]:
size = 20
(swnu_network_umap_20,
    best_swnu_network_umap_20,
    _,
    __,
) = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings.numpy(),
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=dimreduction_method,
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    split_ids=None, 
    split_indices=fold_list,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/18604 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/18604 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in talklife_moc_output/swnu_network_umap_focal_2_20_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in talklife_moc_output/swnu_network_umap_focal_2_20_kfold_best_model.csv


In [20]:
swnu_network_umap_20.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_212489/2570734142.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_20.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.804397   
                                                       0.0005         0.799326   
                                                       0.0010         0.799040   
                                          0.2          0.0001         0.803913   
                                                       0.0005         0.804522   
                                                       0.0010         0.799505   
                           (512, 512)     0.1          0.0001         0.800115   
                                                       0.0005         0.796191   
                                                       0.0010         0.804648   
                                          0.2          0.0001         0.801727   
                                                       0.0005         0.807676   
                                                       0.0010         0.803824   
           (12,)           (256, 256)     0.1          0.0001         0.806457   
                                                       0.0005         0.803573   
                                                       0.0010         0.799846   
                                          0.2          0.0001         0.808106   
                                                       0.0005         0.790493   
                                                       0.0010         0.800688   
                           (512, 512)     0.1          0.0001         0.806583   
                                                       0.0005         0.789328   
                                                       0.0010         0.791604   
                                          0.2          0.0001         0.809288   
                                                       0.0005         0.789006   
                                                       0.0010         0.805293   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.525294   
                                                       0.0005         0.534516   
                                                       0.0010         0.536500   
                                          0.2          0.0001         0.529891   
                                                       0.0005         0.535667   
                                                       0.0010         0.536216   
                           (512, 512)     0.1          0.0001         0.527306   
                                                       0.0005         0.536611   
                                                       0.0010         0.535883   
                                          0.2          0.0001         0.528384   
                                                       0.0005         0.535750   
                                                       0.0010         0.537722   
           (12,)           (256, 256)     0.1          0.0001         0.530316   
                                                       0.0005         0.534207   
                                                       0.0010         0.537429   
                                          0.2          0.0001         0.532395   
                                                       0.0005         0.531036   
                                                       0.0010         0.539575   
                           (512, 512)     0.1          0.0001         0.528839   
                                                       0.0005         0.532850   
                                                       0.0010         0.531237   
                                          0.2      

In [21]:
best_swnu_network_umap_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.802354,0.534371,"[0.8850596966763472, 0.43201803833145436, 0.28...",0.525614,"[0.8971608007326966, 0.39603141794129804, 0.28...",0.545647,"[0.8732806928171166, 0.4751984126984127, 0.288...",None,0.800634,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.793862,0.535776,"[0.879653171653954, 0.4352078239608802, 0.2924...",0.518104,"[0.9010952317350074, 0.39428111155859846, 0.25...",0.560265,"[0.8592078451349975, 0.48561507936507936, 0.33...",None,0.795757,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.800903,0.539354,"[0.8830142223086144, 0.44503078276165353, 0.29...",0.528160,"[0.8987667348150102, 0.39968404423380727, 0.28...",0.554635,"[0.8678043810494142, 0.501984126984127, 0.2941...",None,0.797382,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


In [28]:
best_swnu_network_umap_20[['f1', 'f1_scores', 'precision', 
       'recall', 'valid_f1',
       'valid_f1_scores', 'valid_precision', 
       'valid_recall', 
       'dimensions','swnu_hidden_dim','sig_depth','ffn_hidden_dim', 'dropout_rate', 'learning_rate', 'seed',
       'loss_function', 'k_fold', 
        'batch_size']]

,f1,f1_scores,precision,recall,valid_f1,valid_f1_scores,valid_precision,valid_recall,dimensions,swnu_hidden_dim,sig_depth,ffn_hidden_dim,dropout_rate,learning_rate,seed,loss_function,k_fold,batch_size
0,0.534371,"[0.8850596966763472, 0.43201803833145436, 0.28...",0.525614,0.545647,0.534605,"[0.8833526069686761, 0.4508670520231214, 0.269...",0.526539,0.546919,15,"(10,)",4,"(256, 256)",0.1,0.001,1,focal,True,64
0,0.535776,"[0.879653171653954, 0.4352078239608802, 0.2924...",0.518104,0.560265,0.545512,"[0.8795851445130786, 0.47335686670028576, 0.28...",0.526544,0.572214,15,"(10,)",4,"(256, 256)",0.1,0.001,12,focal,True,64
0,0.539354,"[0.8830142223086144, 0.44503078276165353, 0.29...",0.528160,0.554635,0.537104,"[0.8809222834605857, 0.4595404595404595, 0.270...",0.524361,0.555580,15,"(10,)",4,"(256, 256)",0.1,0.001,123,focal,True,64


In [22]:
best_swnu_network_umap_20["f1"].mean()

0.5365002228247212

In [23]:
best_swnu_network_umap_20["precision"].mean()

0.5239591997687482

In [24]:
best_swnu_network_umap_20["recall"].mean()

0.5535158415831382

In [25]:
np.stack(best_swnu_network_umap_20["f1_scores"]).mean(axis=0)

array([0.8825757 , 0.43741888, 0.28950609])

In [26]:
np.stack(best_swnu_network_umap_20["precision_scores"]).mean(axis=0)

array([0.89900759, 0.39666552, 0.27620449])

In [27]:
np.stack(best_swnu_network_umap_20["recall_scores"]).mean(axis=0)

array([0.86676431, 0.48759921, 0.30618401])